In [1]:
import os
import json

chunks = []

folder = r"/Users/mohamad/Documents/GitHub/Personalized-RAG-Chatbot/chunks.json"

with open(folder, "r", encoding="utf-8") as f:
    data = json.load(f)
    if isinstance(data, list):
        chunks.extend(data)
    elif isinstance(data, dict):
        chunks.append(data)
    else:
        print(f"Skipping {folder} as it is not a list or dictionary.")

print(f"✅ Loaded {len(chunks)} chunks from {folder}")

texts = [c["content"] for c in chunks]
metadata = [
    {
        "id": c["id"],
        "title": c["title"],
        "source": c["source"],
        "text": c["content"]
    }
    for c in chunks
]

✅ Loaded 100 chunks from /Users/mohamad/Documents/GitHub/Personalized-RAG-Chatbot/chunks.json


In [2]:
import openai
import numpy as np
from tqdm import tqdm
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

texts = [c["content"] for c in chunks]
embeddings = []

for text in tqdm(texts, desc="Embedding texts"):
    response = openai.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )
    emb = np.array(response.data[0].embedding, dtype="float32")
    emb /= np.linalg.norm(emb)
    embeddings.append(emb)

embeddings = np.vstack(embeddings)

Embedding texts: 100%|██████████| 100/100 [01:14<00:00,  1.35it/s]


In [3]:
import faiss
import os

dim = embeddings.shape[1]

index = faiss.IndexFlatIP(dim)
index.add(embeddings)

os.makedirs("storage", exist_ok=True)
faiss.write_index(index, "storage/openai_index.faiss")

print(f"✅ FAISS index created and saved with {index.ntotal} vectors.")


✅ FAISS index created and saved with 100 vectors.


In [4]:
def get_embedding(text: str, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model
    )
    embedding = response.data[0].embedding
    return np.array(embedding, dtype='float32')

In [5]:
import json
os.makedirs("storage", exist_ok=True)

metadata = [
    {
        "id": i,
        "title": c.get("title", ""),
        "source": c.get("source", ""),
        "content": c["content"]
    }
    for i, c in enumerate(chunks)
]


with open("storage/chunks_metadata.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

In [46]:
import json
import faiss
import openai

index = faiss.read_index("storage/openai_index.faiss")

with open("storage/chunks_metadata.json", "r", encoding="utf-8") as f:
    metadata = json.load(f)

def search_index(query, k=5, min_score=0.4):
    query_vector = get_embedding(query).reshape(1, -1)
    query_vector /= np.linalg.norm(query_vector)

    distances, indices = index.search(query_vector, k)

    results = []
    for dist, idx in zip(distances[0], indices[0]):
        if dist >= min_score:
            chunk_data = metadata[idx]
            results.append({
                "score": float(dist),
                "chunk": chunk_data["content"],
                "metadata": {
                    "id": chunk_data["id"],
                    "title": chunk_data["title"],
                    "source": chunk_data["source"]
                }
        })
    return results

query = "ما هي العلامات التي تدل على استجابة دعاء الإنسان؟"
query = "كيف يمكن للإنسان أن يعرف أن دعاءه قد استجيب؟"
results = search_index(query, k=6)

print("Top relevant chunks:")
for i, res in enumerate(results, 1):
    print(f"\nResult {i} (score: {res['score']:.4f}):")
    print(res["chunk"])
    print(res["metadata"])


Top relevant chunks:

Result 1 (score: 0.4982):
ثم لا يتغير ولا يتبدل ولا يتأثر ولا ينفعل. ما هذا قلب محجوب وهذا قلب لا يشعر هذه لا يشعر بهذا الدعاء وأهميته بل لا يستطعن طعمه ولذته. لان لذة الدعاء تقربك من الله سبحانه وتعالى. ولذا ورد عندنا ان في بعض بعض من يدعو. بعض الروايات ورد عندنا أن من يدعو الله عز وجل هو يعرف في قرارة نفسه وفي خاتمة دعائه أن دعاؤه وصل أو لم يصل. كيف؟ إذا شعر في قلبه انفعالا. وفي بعض الروايات تتحدث عن قطرة دمع تنزل من عينيه. هذه إشارة على استجابة الدعاء وعلى وصول الدعاء لأن هناك انفعالا ولأن هناك تأثرا على أي حال.
{'id': 56, 'title': 'عاشوراء 2016 - 1438 هجري » كلمة السيد هاشم صفي الدين في الليلة السادسة من شهر محرم الحرام.', 'source': 'موقع السيد هاشم صفي الدين'}

Result 2 (score: 0.4262):
حياتنا هوى. حياتنا شهوة. حياتنا نزوة حياتنا. رغبة حينئذ نحن محجوبون عن الله عز وجل. حينما يغلبنا الهوى. الدعاء لا ينفع. التوسل لا ينفع. الرجوع إلى الله بعد إذن لا ينفع. لا بد أن تقلع عن هذا الهوى حتى يستقيم الأمر. وفي الحديث القدسي إن أدني يقول الله تعالى إن أدني ما أصنع بالعب

In [55]:
def reformulate_query(query):
    model="gpt-5-mini"
    system_prompt = (
        "أنت مساعد متخصص في إعادة صياغة الأسئلة بطريقة مهنية ضمن نظام استرجاع المعلومات (RAG system)."
        "إذا كان السؤال مكتوبًا باللهجة اللبنانية بأحرف إنجليزية، ترجم السؤال إلى العربية الفصحى بأكثر طريقة احترافية ممكنة مع الحفاظ على الاحترافية في التعبير، مع الأخذ بعين الاعتبار سياق المحادثة وسيرها السابق للتحقق من توافق السؤال مع مجرى الحوار."
        "ابدأ بخطة مختصرة (Checklist) من 3-5 خطوات مفاهيمية لكل مرحلة تعالج فيها السؤال."
        "أعد كتابة السؤال بنفس الصيغة المستخدمة من قبل المتكلم (لا تغيّر الضمائر أو وجهة النظر)."
        "لا تضف أو تحذف أي معنى جديد."
        "إذا كان السؤال واضحًا ومباشرًا، أعد عرضه كما هو مع تحسين طفيف للأسلوب فقط."
        "الهدف هو جعل السؤال أوضح وأكثر رسمية دون تغيير معناه أو صيغة المتكلم."
        "بعد تعديل كل سؤال، تحقق في جملة أو جملتين من أن التعديل حقق الوضوح والاحترافية دون تغيير الجوهر، ثم انتقل للسؤال التالي."
        "إذا لم يتوافق السؤال مع ما قبل، أعد صياغته مرة ثانية وتحقق مرة أخرى من توافقه مع مجرى الحوار."
        "اكتب فقط الصيغة النهائية للسؤال من دون الشرح والخطوات."
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=1,
    )
    return response.choices[0].message.content.strip()
    

In [8]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def generate_answer(query, retrieved_chunks, model="gpt-4o-mini"):

    context = "\n\n".join([c["chunk"] for c in retrieved_chunks])

    messages = [
        {
            "role": "system",
            "content": (
                "أنت مساعد يجيب على انه السيد هاشم صفي الدين."
                "اعتمد في اجابتك على النصوص المتوفرة في السياق فقط."
                "إذا لم يكن الجواب واضحا وكاملا في السياق، قل أنك لا تعرف. "
                "تكلم باحترام عن الشخصيات الشيعية, مع ذكر الألقاب المناسبة."
                "أجب دائمًا باللغة العربية الفصحى الواضحة."
            ),
        },
        {
            "role": "user",
            "content": f"السياق:\n{context}\n\nالسؤال: {query}",
        },
    ]

    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.2,
    )

    return response.choices[0].message.content.strip()


In [47]:
query = "ما هي العلامات التي تدل على استجابة دعاء الإنسان؟"
query = "كيف يمكن للإنسان أن يعرف أن دعاءه قد استجيب؟"
query = "ما هو رأيك فيما حدث بالأمس؟"

results = search_index(query, k=3)

retrieved_chunks = results
answer = generate_answer(query, retrieved_chunks)
print(answer)

عذرًا، لا أستطيع تقديم رأي حول ما حدث بالأمس، حيث لا تتوفر لدي معلومات كافية عن الحدث المعني.


In [54]:
history = "{'role': 'system', 'content': '\n\nالرسائل السابقة:\nuser: aan shu btaaref tehke'}, {'role': 'user', 'content': 'السياق:\n\n\n\nالسؤال: عن ماذا تستطيع التحدث؟'}]"
query = "ktebun mra2amen"
# query =  "سلام"

refined_query = reformulate_query(query)
print("🔄 Reformulated:", refined_query)


🔄 Reformulated: قائمة التحقق (Checklist):
1. تحديد اللهجة والنية: ترجمة العبارة من اللهجة اللبنانية المكتوبة باللاتينية إلى العربية الفصحى.
2. الحفاظ على صيغة المتكلم والضمائر: الإبقاء على صيغة الأمر للمخاطبين كما وردت.
3. ضبط الأسلوب ليكون رسميًا وواضحًا دون تغيير المعنى.
4. مراجعة للتأكد من الوضوح والاحترافية.

اكتبوا مُرقَّمًا
